<a href="https://colab.research.google.com/github/AhmedAboulezz/Trading/blob/main/Divergence_Strategy_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages
!pip install pandas numpy matplotlib plotly scipy ta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema
import ta
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#------------------------------------------------------------------------------
# CONFIGURATION PARAMETERS
#------------------------------------------------------------------------------

class Config:
    # Pivot settings
    prd = 5  # Pivot Period
    source = "Close"  # Source for Pivot Points: "Close" or "High/Low"

    # Divergence settings
    searchdiv = "Regular"  # "Regular", "Hidden", or "Regular/Hidden"
    showlimit = 1  # Minimum Number of Divergence (any)
    maxpp = 10  # Maximum Pivot Points to Check
    maxbars = 100  # Maximum Bars to Check
    dontconfirm = False  # Don't Wait for Confirmation

    # Quality filter
    minPosDivForEntry = 2  # Min # of Positive Divergence for Valid Long
    minNegDivForExit = 1   # Min # of Negative Divergence for Exit

    # Entry delay and drawdown
    delayBars = 0  # Delay For Entry (Bars)
    useDrawdown = False  # Use % Drawdown Before Entry
    drawdownPerc = 1.0  # Drawdown %

    # Indicator selection (all enabled by default)
    calcmacd = True
    calcmacda = True
    calcrsi = True
    calcstoc = True
    calccci = True
    calcmom = True
    calcobv = False
    calcvwmacd = True
    calccmf = True
    calcmfi = True

    # Backtest settings
    initial_capital = 10000
    position_size = 1.0  # Use 100% of capital per trade

config = Config()

#------------------------------------------------------------------------------
# DATA LOADING
#------------------------------------------------------------------------------

def load_data_from_csv(filepath):
    """Load OHLC data from CSV file"""
    print(f"Loading data from {filepath}...")

    # Read CSV
    df = pd.read_csv(filepath)

    # Normalize column names to lowercase
    df.columns = [col.lower().strip() for col in df.columns]

    # Parse time column
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)

    # Ensure we have required columns
    required_cols = ['open', 'high', 'low', 'close']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Add volume if not present (needed for some indicators)
    if 'volume' not in df.columns:
        df['volume'] = 1000000  # Default volume
        print("Warning: No volume column found, using default values")

    print(f"Loaded {len(df)} bars from {df.index[0]} to {df.index[-1]}")
    return df

#------------------------------------------------------------------------------
# TECHNICAL INDICATORS
#------------------------------------------------------------------------------

def calculate_indicators(df):
    df = df.copy()

    close = df["close"].astype(float)
    high  = df["high"].astype(float)
    low   = df["low"].astype(float)
    vol   = df["volume"].astype(float)  # أو "volume" حسب اسم العمود عندك

    # ------------------------------------------------------------------
    # RSI, MACD, VW-MACD, CMF, ... خليهـم زي ما أنت عاملهم لو متطابقين
    # ------------------------------------------------------------------
    import ta

    # RSI(14) - مطابق لـ ta.rsi(close, 14)
    df["rsi"] = ta.momentum.RSIIndicator(close=close, window=14).rsi()

    # MACD(12,26,9)
    macd_ind = ta.trend.MACD(close=close, window_slow=26,
                             window_fast=12, window_sign=9)
    df["macd"]        = macd_ind.macd()
    df["macd_signal"] = macd_ind.macd_signal()
    df["macd_hist"]   = macd_ind.macd_diff()

    # Momentum(10)
    df["momentum"] = close - close.shift(10)

    # VW-MACD = VWMA(12) - VWMA(26) (نفس Pine)
    vwma_fast = (close * vol).rolling(12).sum() / vol.rolling(12).sum()
    vwma_slow = (close * vol).rolling(26).sum() / vol.rolling(26).sum()
    df["vwmacd"] = vwma_fast - vwma_slow

    # CMF(21) زي Pine
    df["cmf"] = ta.volume.ChaikinMoneyFlowIndicator(
        high=high, low=low, close=close, volume=vol, window=21
    ).chaikin_money_flow()

    # ==============================================================
    # ✅ 1) Stochastic(14,3) مطابق لـ Stochastic(14,3) في Pine
    # ==============================================================
    lowest_low_14  = low.rolling(window=14, min_periods=14).min()
    highest_high_14 = high.rolling(window=14, min_periods=14).max()

    stoch_k = 100.0 * (close - lowest_low_14) / (highest_high_14 - lowest_low_14)
    df["stoch"] = stoch_k.rolling(window=3, min_periods=3).mean()

    # ==============================================================
    # ✅ 2) CCI(10) باستخدام الـ close (نفس Pine في csv)
    # ==============================================================
    n_cci = 10
    sma_close = close.rolling(n_cci, min_periods=n_cci).mean()

    # mean absolute deviation من الـ sma على الـ close
    mad_close = close.rolling(n_cci, min_periods=n_cci).apply(
        lambda x: np.mean(np.abs(x - x.mean())),
        raw=True
    )

    df["cci"] = (close - sma_close) / (0.015 * mad_close)

    # ==============================================================
    # ✅ 3) MFI(14) باستخدام الـ close كـ price
    # ==============================================================
    n_mfi = 14
    price = close

    raw_mf = price * vol
    pos_mf = [0.0]
    neg_mf = [0.0]

    for i in range(1, len(df)):
        if price.iloc[i] > price.iloc[i - 1]:
            pos_mf.append(raw_mf.iloc[i])
            neg_mf.append(0.0)
        elif price.iloc[i] < price.iloc[i - 1]:
            pos_mf.append(0.0)
            neg_mf.append(raw_mf.iloc[i])
        else:
            pos_mf.append(0.0)
            neg_mf.append(0.0)

    pos_mf = pd.Series(pos_mf, index=df.index)
    neg_mf = pd.Series(neg_mf, index=df.index)

    pos_roll = pos_mf.rolling(n_mfi, min_periods=n_mfi).sum()
    neg_roll = neg_mf.rolling(n_mfi, min_periods=n_mfi).sum()

    mfi = 100.0 * (pos_roll / (pos_roll + neg_roll))
    df["mfi"] = mfi

    # 9) OBV (هنسيبه زي ما انت عامل)
    obv_values = [0.0]
    for i in range(1, len(df)):
        if close.iloc[i] > close.iloc[i-1]:
            obv_values.append(obv_values[-1] + vol.iloc[i])
        elif close.iloc[i] < close.iloc[i-1]:
            obv_values.append(obv_values[-1] - vol.iloc[i])
        else:
            obv_values.append(obv_values[-1])
    df['obv'] = obv_values

    df = df.fillna(method='bfill').fillna(method='ffill')

    # ==============================================================
    # OBV زي ما تحب (وانت أصلاً قلت سيبه)
    # ==============================================================
    # ممكن تخليه زي Pine لو حابب لاحقًا، بس مش ضروري دلوقتي

    return df
#------------------------------------------------------------------------------
# PIVOT POINT DETECTION
#------------------------------------------------------------------------------

def find_pivots(df, period):
    """Find pivot highs and pivot lows"""
    print(f"Finding pivot points with period {period}...")

    if config.source == "Close":
        high_series = df['close']
        low_series = df['close']
    else:
        high_series = df['high']
        low_series = df['low']

    # Initialize pivot columns
    df['pivot_high'] = np.nan
    df['pivot_high_bar'] = np.nan
    df['pivot_low'] = np.nan
    df['pivot_low_bar'] = np.nan

    for i in range(period, len(df) - period):
        # Check pivot high
        is_pivot_high = True
        for j in range(1, period + 1):
            if high_series.iloc[i] <= high_series.iloc[i - j] or \
               high_series.iloc[i] <= high_series.iloc[i + j]:
                is_pivot_high = False
                break

        if is_pivot_high:
            df.iloc[i, df.columns.get_loc('pivot_high')] = high_series.iloc[i]
            df.iloc[i, df.columns.get_loc('pivot_high_bar')] = i

        # Check pivot low
        is_pivot_low = True
        for j in range(1, period + 1):
            if low_series.iloc[i] >= low_series.iloc[i - j] or \
               low_series.iloc[i] >= low_series.iloc[i + j]:
                is_pivot_low = False
                break

        if is_pivot_low:
            df.iloc[i, df.columns.get_loc('pivot_low')] = low_series.iloc[i]
            df.iloc[i, df.columns.get_loc('pivot_low_bar')] = i

    num_highs = df['pivot_high'].notna().sum()
    num_lows = df['pivot_low'].notna().sum()
    print(f"Found {num_highs} pivot highs and {num_lows} pivot lows")

    return df

#------------------------------------------------------------------------------
# DIVERGENCE DETECTION
#------------------------------------------------------------------------------

def detect_divergence(df, indicator_name, bar_idx, pivot_positions, pivot_values,
                     is_bullish, is_regular):
    """
    Detect divergence at a specific bar
    Returns: length of divergence if found, else 0
    """

    if bar_idx < config.prd:
        return 0

    startpoint = 0 if config.dontconfirm else 1

    if config.source == "Close":
        price_series = df['close'].values
    else:
        price_series = df['low'].values if is_bullish else df['high'].values

    indicator_series = df[indicator_name].values

    # Check confirmation condition
    if not config.dontconfirm:
        if is_bullish:
            if not (indicator_series[bar_idx] > indicator_series[bar_idx - 1] or
                   df['close'].values[bar_idx] > df['close'].values[bar_idx - 1]):
                return 0
        else:
            if not (indicator_series[bar_idx] < indicator_series[bar_idx - 1] or
                   df['close'].values[bar_idx] < df['close'].values[bar_idx - 1]):
                return 0

    # Check each pivot
    for pivot_idx in range(min(config.maxpp, len(pivot_positions))):
        if pivot_idx >= len(pivot_positions) or np.isnan(pivot_positions[pivot_idx]):
            break

        pivot_bar = int(pivot_positions[pivot_idx])
        length = bar_idx - pivot_bar

        if length > config.maxbars:
            break

        if length > 5:
            # Check divergence conditions
            if is_bullish and is_regular:
                # Positive Regular: indicator makes higher low, price makes lower low
                div_condition = (indicator_series[bar_idx - startpoint] > indicator_series[pivot_bar] and
                               price_series[bar_idx - startpoint] < pivot_values[pivot_idx])
            elif is_bullish and not is_regular:
                # Positive Hidden: indicator makes lower low, price makes higher low
                div_condition = (indicator_series[bar_idx - startpoint] < indicator_series[pivot_bar] and
                               price_series[bar_idx - startpoint] > pivot_values[pivot_idx])
            elif not is_bullish and is_regular:
                # Negative Regular: indicator makes lower high, price makes higher high
                div_condition = (indicator_series[bar_idx - startpoint] < indicator_series[pivot_bar] and
                               price_series[bar_idx - startpoint] > pivot_values[pivot_idx])
            else:
                # Negative Hidden: indicator makes higher high, price makes lower high
                div_condition = (indicator_series[bar_idx - startpoint] > indicator_series[pivot_bar] and
                               price_series[bar_idx - startpoint] < pivot_values[pivot_idx])

            if div_condition:
                # Check if line is valid (no crossings)
                slope1 = (indicator_series[bar_idx - startpoint] - indicator_series[pivot_bar]) / length
                slope2 = (df['close'].values[bar_idx - startpoint] - df['close'].values[pivot_bar]) / length

                virtual_line1 = indicator_series[bar_idx - startpoint]
                virtual_line2 = df['close'].values[bar_idx - startpoint]

                valid = True
                for y in range(1 + startpoint, length):
                    virtual_line1 -= slope1
                    virtual_line2 -= slope2

                    check_idx = bar_idx - y
                    if is_bullish:
                        if indicator_series[check_idx] < virtual_line1 or \
                           df['close'].values[check_idx] < virtual_line2:
                            valid = False
                            break
                    else:
                        if indicator_series[check_idx] > virtual_line1 or \
                           df['close'].values[check_idx] > virtual_line2:
                            valid = False
                            break

                if valid:
                    return length

    return 0

def scan_all_divergences(df):
    """Scan for all divergences across all indicators"""
    print("Scanning for divergences...")

    n = len(df)

    # ==============================
    # 1) اختار المؤشرات زي ما كنت عامل
    # ==============================
    indicators_to_check = []

    if config.calcmacd:
        indicators_to_check.append('macd')
    if config.calcmacda:
        indicators_to_check.append('macd_hist')
    if config.calcrsi:
        indicators_to_check.append('rsi')
    if config.calcstoc:
        indicators_to_check.append('stoch')
    if config.calccci:
        indicators_to_check.append('cci')
    if config.calcmom:
        indicators_to_check.append('momentum')
    if config.calcobv:
        indicators_to_check.append('obv')
    if config.calcvwmacd:
        indicators_to_check.append('vwmacd')
    if config.calccmf:
        indicators_to_check.append('cmf')
    if config.calcmfi:
        indicators_to_check.append('mfi')

    # ==============================
    # 2) جهّز أعمدة الديفرجنس كـ numpy arrays
    #    بدل ما نكتب في df جوّه اللوب
    # ==============================
    div_cols = []
    div_arrays = {}

    for ind in indicators_to_check:
        for div_type in ['pos_reg', 'neg_reg', 'pos_hid', 'neg_hid']:
            col_name = f'{ind}_{div_type}'
            div_cols.append(col_name)
            # نفس الفكرة: 0 = مفيش ديفرجنس، رقم = length
            div_arrays[col_name] = np.zeros(n, dtype=np.int32)

    # ==============================
    # 3) خُد الـ values مرّة واحدة
    # ==============================
    pivot_high_vals_all = df['pivot_high'].values
    pivot_low_vals_all  = df['pivot_low'].values

    indicator_values = {
        ind: df[ind].values
        for ind in indicators_to_check
    }

    # ==============================
    # 4) اللوب الرئيسي على البارات
    # ==============================
    for i in range(config.prd + 10, n):
        # ---------------------------------------
        # جمع آخر pivot highs / lows قبل البار ده
        # ---------------------------------------
        pivot_high_bars = []
        pivot_high_vals = []
        pivot_low_bars = []
        pivot_low_vals = []

        start_j = i - 1
        end_j = max(0, i - config.maxbars)

        # Pivot Highs
        for j in range(start_j, end_j, -1):
            ph = pivot_high_vals_all[j]
            if not np.isnan(ph):
                pivot_high_bars.append(j)
                pivot_high_vals.append(ph)
                if len(pivot_high_bars) >= config.maxpp:
                    break

        # Pivot Lows
        for j in range(start_j, end_j, -1):
            pl = pivot_low_vals_all[j]
            if not np.isnan(pl):
                pivot_low_bars.append(j)
                pivot_low_vals.append(pl)
                if len(pivot_low_bars) >= config.maxpp:
                    break

        # ---------------------------------------
        # دلوقتي نعدّي على المؤشرات وننده detect_divergence
        # ---------------------------------------
        for ind in indicators_to_check:
            ind_arr = indicator_values[ind]

            # نفس check القديم: لو مفيش قيمة valid هنا سيبه
            if np.isnan(ind_arr[i]):
                continue

            # Positive Regular Divergence (bullish, regular)
            if config.searchdiv in ["Regular", "Regular/Hidden"] and len(pivot_low_bars) > 0:
                div_len = detect_divergence(
                    df, ind, i,
                    pivot_low_bars, pivot_low_vals,
                    is_bullish=True, is_regular=True
                )
                if div_len > 0:
                    div_arrays[f'{ind}_pos_reg'][i] = div_len

            # Negative Regular Divergence (bearish, regular)
            if config.searchdiv in ["Regular", "Regular/Hidden"] and len(pivot_high_bars) > 0:
                div_len = detect_divergence(
                    df, ind, i,
                    pivot_high_bars, pivot_high_vals,
                    is_bullish=False, is_regular=True
                )
                if div_len > 0:
                    div_arrays[f'{ind}_neg_reg'][i] = div_len

            # Positive Hidden Divergence (bullish, hidden)
            if config.searchdiv in ["Hidden", "Regular/Hidden"] and len(pivot_low_bars) > 0:
                div_len = detect_divergence(
                    df, ind, i,
                    pivot_low_bars, pivot_low_vals,
                    is_bullish=True, is_regular=False
                )
                if div_len > 0:
                    div_arrays[f'{ind}_pos_hid'][i] = div_len

            # Negative Hidden Divergence (bearish, hidden)
            if config.searchdiv in ["Hidden", "Regular/Hidden"] and len(pivot_high_bars) > 0:
                div_len = detect_divergence(
                    df, ind, i,
                    pivot_high_bars, pivot_high_vals,
                    is_bullish=False, is_regular=False
                )
                if div_len > 0:
                    div_arrays[f'{ind}_neg_hid'][i] = div_len

    # ==============================
    # 5) بعد ما خلّصنا، رجّع الأعمدة لـ df مرة واحدة
    # ==============================
    for col_name in div_cols:
        df[col_name] = div_arrays[col_name]

    # ==============================
    # 6) نفس الكود القديم: total_divergences + filter + counts
    # ==============================
    # Count total divergences
    df['total_divergences'] = 0
    for col in div_cols:
        df['total_divergences'] += (df[col] > 0).astype(int)

    # Filter by minimum divergences
    df.loc[df['total_divergences'] < config.showlimit, div_cols] = 0

    # Count positive and negative divergences
    df['pos_div_count'] = 0
    df['neg_div_count'] = 0

    for col in div_cols:
        if 'pos_' in col:
            df['pos_div_count'] += (df[col] > 0).astype(int)
        else:
            df['neg_div_count'] += (df[col] > 0).astype(int)

    total_divs = (df['total_divergences'] > 0).sum()
    print(f"Found {total_divs} bars with divergences")

    return df

#------------------------------------------------------------------------------
# TRADING STRATEGY
#------------------------------------------------------------------------------

def run_strategy(df):
    """Execute trading strategy based on divergences"""
    print("Running strategy...")

    df['signal'] = 0
    df['position'] = 0
    df['entry_price'] = np.nan
    df['exit_price'] = np.nan

    buy_signal_bar = None
    buy_signal_price = None
    sell_signal_bar = None  # ✅ Track when we get exit signal
    in_position = False

    for i in range(len(df)):
        # Check for entry signal (positive divergences >= threshold)
        if not in_position and df['pos_div_count'].iloc[i] >= config.minPosDivForEntry:
            buy_signal_bar = i
            buy_signal_price = df['close'].iloc[i]

        # Process entry after delay
        if buy_signal_bar is not None and not in_position:
            bars_since_signal = i - buy_signal_bar

            # Need at least 1 bar delay to enter at next bar's open
            if bars_since_signal >= max(1, config.delayBars):
                enter_trade = True

                if config.useDrawdown:
                    cur_drawdown = (buy_signal_price - df['close'].iloc[i-1]) / buy_signal_price * 100.0
                    if cur_drawdown < config.drawdownPerc:
                        enter_trade = False

                if enter_trade:
                    df.loc[df.index[i], 'signal'] = 1
                    df.loc[df.index[i], 'entry_price'] = df['open'].iloc[i]  # Next bar's open
                    in_position = True
                    buy_signal_bar = None
                    buy_signal_price = None

        # Set position status
        if in_position:
            df.loc[df.index[i], 'position'] = 1

        # ✅ Check for exit signal (negative divergences >= threshold)
        if in_position and df['neg_div_count'].iloc[i] >= config.minNegDivForExit:
            sell_signal_bar = i  # Mark the signal bar

        # ✅ Process exit at NEXT bar's open (just like entry)
        if sell_signal_bar is not None and in_position:
            bars_since_exit_signal = i - sell_signal_bar

            # Exit at next bar's open (1 bar delay)
            if bars_since_exit_signal >= 1:
                df.loc[df.index[i], 'signal'] = -1
                df.loc[df.index[i], 'exit_price'] = df['open'].iloc[i]  # Next bar's open
                in_position = False
                sell_signal_bar = None

    return df

#------------------------------------------------------------------------------
# VISUALIZATION
#------------------------------------------------------------------------------

def plot_candlestick_with_signals(df):
    """Create interactive candlestick chart with entry/exit signals"""
    print("Creating candlestick chart...")

    # Create figure with subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=('Price with Entry/Exit Signals', 'Divergence Counts'),
        row_heights=[0.7, 0.3]
    )

    # Add candlestick
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Price',
            increasing_line_color='green',
            decreasing_line_color='red'
        ),
        row=1, col=1
    )

    # Add entry signals (buy)
    entries = df[df['signal'] == 1]
    if len(entries) > 0:
        fig.add_trace(
            go.Scatter(
                x=entries.index,
                y=entries['entry_price'],
                mode='markers',
                name='BUY',
                marker=dict(
                    symbol='triangle-up',
                    size=15,
                    color='lime',
                    line=dict(color='darkgreen', width=2)
                )
            ),
            row=1, col=1
        )

    # Add exit signals (sell)
    exits = df[df['signal'] == -1]
    if len(exits) > 0:
        fig.add_trace(
            go.Scatter(
                x=exits.index,
                y=exits['exit_price'],
                mode='markers',
                name='SELL',
                marker=dict(
                    symbol='triangle-down',
                    size=15,
                    color='red',
                    line=dict(color='darkred', width=2)
                )
            ),
            row=1, col=1
        )

    # Add positive divergence count
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['pos_div_count'],
            mode='lines',
            name='Positive Divergences',
            line=dict(color='green', width=1),
            fill='tozeroy'
        ),
        row=2, col=1
    )

    # Add negative divergence count
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['neg_div_count'],
            mode='lines',
            name='Negative Divergences',
            line=dict(color='red', width=1),
            fill='tozeroy'
        ),
        row=2, col=1
    )

    # Add threshold lines
    fig.add_hline(
        y=config.minPosDivForEntry,
        line_dash="dash",
        line_color="green",
        annotation_text=f"Entry Threshold ({config.minPosDivForEntry})",
        row=2, col=1
    )

    fig.add_hline(
        y=config.minNegDivForExit,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Exit Threshold ({config.minNegDivForExit})",
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        title='Divergence-Based Trading Strategy',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis2_title='Time',
        yaxis2_title='Divergence Count',
        height=900,
        showlegend=True,
        xaxis_rangeslider_visible=False
    )

    fig.show()
    print("Chart displayed successfully")

#------------------------------------------------------------------------------
# PERFORMANCE METRICS
#------------------------------------------------------------------------------

def calculate_performance(df):
    """Calculate and display performance metrics"""
    print("\n" + "="*60)
    print("PERFORMANCE METRICS")
    print("="*60)

    entries = df[df['signal'] == 1].copy()
    exits = df[df['signal'] == -1].copy()

    if len(entries) == 0:
        print("No trades were executed")
        return

    # Match entries with exits
    trades = []
    for i, entry_row in entries.iterrows():
        entry_idx = df.index.get_loc(i)
        entry_price = entry_row['entry_price']

        # Find next exit
        exit_found = False
        for j, exit_row in exits.iterrows():
            exit_idx = df.index.get_loc(j)
            if exit_idx > entry_idx:
                exit_price = exit_row['exit_price']
                pnl_pct = (exit_price - entry_price) / entry_price * 100
                trades.append({
                    'entry_time': i,
                    'exit_time': j,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'pnl_pct': pnl_pct,
                    'bars_held': exit_idx - entry_idx,
                    'entry_idx': entry_idx  # Store index for indicator lookup
                })
                exit_found = True
                break

        if not exit_found:
            # Still in trade
            trades.append({
                'entry_time': i,
                'exit_time': None,
                'entry_price': entry_price,
                'exit_price': df['close'].iloc[-1],
                'pnl_pct': (df['close'].iloc[-1] - entry_price) / entry_price * 100,
                'bars_held': len(df) - entry_idx - 1,
                'entry_idx': entry_idx
            })

    trades_df = pd.DataFrame(trades)

    # Calculate metrics
    total_trades = len(trades_df)
    winning_trades = len(trades_df[trades_df['pnl_pct'] > 0])
    losing_trades = len(trades_df[trades_df['pnl_pct'] < 0])
    win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
    avg_win = trades_df[trades_df['pnl_pct'] > 0]['pnl_pct'].mean() if winning_trades > 0 else 0
    avg_loss = trades_df[trades_df['pnl_pct'] < 0]['pnl_pct'].mean() if losing_trades > 0 else 0
    avg_pnl = trades_df['pnl_pct'].mean()
    total_return = trades_df['pnl_pct'].sum()
    avg_bars_held = trades_df['bars_held'].mean()

    # Display metrics
    print(f"Total Trades:       {total_trades}")
    print(f"Winning Trades:     {winning_trades}")
    print(f"Losing Trades:      {losing_trades}")
    print(f"Win Rate:           {win_rate:.2f}%")
    print(f"Average Win:        {avg_win:.2f}%")
    print(f"Average Loss:       {avg_loss:.2f}%")
    print(f"Average P&L:        {avg_pnl:.2f}%")
    print(f"Total Return:       {total_return:.2f}%")
    print(f"Avg Bars Held:      {avg_bars_held:.1f}")

    print("\n" + "="*60)

    # Display trade list with indicator values
    print("\nDETAILED TRADE ANALYSIS WITH INDICATORS:")
    print("="*60)

    for idx, trade in trades_df.iterrows():
        status = "CLOSED" if trade['exit_time'] is not None else "OPEN"
        entry_idx = trade['entry_idx']

        print(f"\n{'='*60}")
        print(f"Trade #{idx+1} [{status}]")
        print(f"{'='*60}")

        # Price information
        print(f"\n📊 PRICE INFORMATION:")
        print(f"  Entry Time:      {trade['entry_time']}")
        print(f"  Entry Price:     ${trade['entry_price']:.2f}")
        print(f"  Open:            ${df['open'].iloc[entry_idx]:.2f}")
        print(f"  High:            ${df['high'].iloc[entry_idx]:.2f}")
        print(f"  Low:             ${df['low'].iloc[entry_idx]:.2f}")
        print(f"  Close:           ${df['close'].iloc[entry_idx]:.2f}")

        if trade['exit_time'] is not None:
            print(f"  Exit Time:       {trade['exit_time']}")
            print(f"  Exit Price:      ${trade['exit_price']:.2f}")
        else:
            print(f"  Current Price:   ${trade['exit_price']:.2f}")

        print(f"  P&L:             {trade['pnl_pct']:+.2f}%")
        print(f"  Duration:        {int(trade['bars_held'])} bars")

        # Divergence counts
        print(f"\n🔍 DIVERGENCE SIGNALS:")
        print(f"  Positive Divs:   {int(df['pos_div_count'].iloc[entry_idx])}")
        print(f"  Negative Divs:   {int(df['neg_div_count'].iloc[entry_idx])}")
        print(f"  Total Divs:      {int(df['total_divergences'].iloc[entry_idx])}")

        # Technical indicators
        print(f"\n📈 TECHNICAL INDICATORS AT ENTRY:")

        # Momentum indicators
        if config.calcrsi:
            print(f"  RSI (14):        {df['rsi'].iloc[entry_idx]:.2f}")

        if config.calcstoc:
            print(f"  Stochastic:      {df['stoch'].iloc[entry_idx]:.2f}")

        if config.calcmom:
            print(f"  Momentum (10):   {df['momentum'].iloc[entry_idx]:.2f}")

        if config.calccci:
            print(f"  CCI (10):        {df['cci'].iloc[entry_idx]:.2f}")

        # Trend indicators
        if config.calcmacd:
            print(f"  MACD:            {df['macd'].iloc[entry_idx]:.4f}")
            print(f"  MACD Signal:     {df['macd_signal'].iloc[entry_idx]:.4f}")

        if config.calcmacda:
            print(f"  MACD Histogram:  {df['macd_hist'].iloc[entry_idx]:.4f}")

        # Volume indicators
        if config.calcobv:
            print(f"  OBV:             {df['obv'].iloc[entry_idx]:,.0f}")

        if config.calcvwmacd:
            print(f"  VWMACD:          {df['vwmacd'].iloc[entry_idx]:.4f}")

        if config.calccmf:
            print(f"  CMF (21):        {df['cmf'].iloc[entry_idx]:.4f}")

        if config.calcmfi:
            print(f"  MFI (14):        {df['mfi'].iloc[entry_idx]:.2f}")

        # Individual divergences detected
        print(f"\n🎯 SPECIFIC DIVERGENCES DETECTED:")
        divergence_found = False

        # Check each indicator's divergences
        indicators_list = []
        if config.calcmacd:
            indicators_list.append(('MACD', 'macd'))
        if config.calcmacda:
            indicators_list.append(('MACD Hist', 'macd_hist'))
        if config.calcrsi:
            indicators_list.append(('RSI', 'rsi'))
        if config.calcstoc:
            indicators_list.append(('Stochastic', 'stoch'))
        if config.calccci:
            indicators_list.append(('CCI', 'cci'))
        if config.calcmom:
            indicators_list.append(('Momentum', 'momentum'))
        if config.calcobv:
            indicators_list.append(('OBV', 'obv'))
        if config.calcvwmacd:
            indicators_list.append(('VWMACD', 'vwmacd'))
        if config.calccmf:
            indicators_list.append(('CMF', 'cmf'))
        if config.calcmfi:
            indicators_list.append(('MFI', 'mfi'))

        for ind_name, ind_col in indicators_list:
            divs = []
            if f'{ind_col}_pos_reg' in df.columns and df[f'{ind_col}_pos_reg'].iloc[entry_idx] > 0:
                divs.append(f"Pos Regular ({int(df[f'{ind_col}_pos_reg'].iloc[entry_idx])} bars)")
                divergence_found = True
            if f'{ind_col}_neg_reg' in df.columns and df[f'{ind_col}_neg_reg'].iloc[entry_idx] > 0:
                divs.append(f"Neg Regular ({int(df[f'{ind_col}_neg_reg'].iloc[entry_idx])} bars)")
                divergence_found = True
            if f'{ind_col}_pos_hid' in df.columns and df[f'{ind_col}_pos_hid'].iloc[entry_idx] > 0:
                divs.append(f"Pos Hidden ({int(df[f'{ind_col}_pos_hid'].iloc[entry_idx])} bars)")
                divergence_found = True
            if f'{ind_col}_neg_hid' in df.columns and df[f'{ind_col}_neg_hid'].iloc[entry_idx] > 0:
                divs.append(f"Neg Hidden ({int(df[f'{ind_col}_neg_hid'].iloc[entry_idx])} bars)")
                divergence_found = True

            if divs:
                print(f"  {ind_name:15s}: {', '.join(divs)}")

        if not divergence_found:
            print(f"  No specific divergences recorded (signal from previous bar)")

    print("\n" + "="*60)

#------------------------------------------------------------------------------
# MAIN EXECUTION
#------------------------------------------------------------------------------

def main(csv_filepath):
    """Main execution function"""

    print("="*60)
    print("DIVERGENCE-BASED TRADING STRATEGY")
    print("="*60)
    print(f"\nConfiguration:")
    print(f"  Pivot Period: {config.prd}")
    print(f"  Divergence Type: {config.searchdiv}")
    print(f"  Min Positive Divergences for Entry: {config.minPosDivForEntry}")
    print(f"  Min Negative Divergences for Exit: {config.minNegDivForExit}")
    print(f"  Entry Delay: {config.delayBars} bars")
    print("\n" + "="*60 + "\n")

    # Load data
    df = load_data_from_csv(csv_filepath)

    # Calculate indicators
    df = calculate_indicators(df)

    # Find pivot points
    df = find_pivots(df, config.prd)

    # Scan for divergences
    df = scan_all_divergences(df)

    # Run strategy
    df = run_strategy(df)

    # Calculate performance
    calculate_performance(df)

    # Plot results
    # plot_candlestick_with_signals(df)

    return df

# ============================================================================
# RUN THE STRATEGY
# ============================================================================

# Upload your CSV file to Colab first, then specify the path
# Example: df_result = main('/content/your_data.csv')

# For Google Colab, you can upload a file like this:
# from google.colab import files
# uploaded = files.upload()

# Get the uploaded filename
csv_filename = 'PLD.csv'

# Run the strategy
df_result = main(csv_filename)

Streaming output truncated to the last 5000 lines.
  Open:            $984.51
  High:            $984.71
  Low:             $982.24
  Close:           $982.81
  Exit Time:       2025-03-13 19:45:00-04:00
  Exit Price:      $995.41
  P&L:             +1.11%
  Duration:        76 bars

🔍 DIVERGENCE SIGNALS:
  Positive Divs:   0
  Negative Divs:   0
  Total Divs:      0

📈 TECHNICAL INDICATORS AT ENTRY:
  RSI (14):        36.49
  Stochastic:      10.72
  Momentum (10):   -4.10
  CCI (10):        -142.62
  MACD:            -0.5217
  MACD Signal:     -0.0807
  MACD Histogram:  -0.4410
  VWMACD:          -0.1737
  CMF (21):        -0.0370
  MFI (14):        41.61

🎯 SPECIFIC DIVERGENCES DETECTED:
  No specific divergences recorded (signal from previous bar)

Trade #38 [CLOSED]

📊 PRICE INFORMATION:
  Entry Time:      2025-03-14 10:45:00-04:00
  Entry Price:     $993.19
  Open:            $993.19
  High:            $993.32
  Low:             $989.62
  Close:           $990.62
  Exit Time:    

Chart displayed successfully


In [4]:
#===============================================================================
# CELL 2: Auto "window + indicators + pivots" search (with drawdown stats)
#===============================================================================

!pip install scikit-learn

from sklearn.model_selection import ParameterGrid

#-------------------------------------------
# 1) Parametric indicator calculator
#-------------------------------------------

def calculate_indicators_with_params(df, params):
    """
    نسخة من calculate_indicators لكن بالـ windows جاية من params
    ما بنلمسش Config ولا الاستراتيجية نفسها، بس بنغير إعدادات المؤشرات.
    """
    df = df.copy()

    # نقرأ الـ windows من params مع قيم default
    rsi_window      = params.get("rsi_window",      14)
    macd_fast       = params.get("macd_fast",       12)
    macd_slow       = params.get("macd_slow",       26)
    macd_signal     = params.get("macd_signal",      9)
    mom_window      = params.get("mom_window",      10)
    stoch_window    = params.get("stoch_window",    14)
    stoch_smooth    = params.get("stoch_smooth",     3)
    vwma_fast_win   = params.get("vwma_fast",       12)
    vwma_slow_win   = params.get("vwma_slow",       26)
    cmf_window      = params.get("cmf_window",      21)
    cci_window      = params.get("cci_window",      10)
    mfi_window      = params.get("mfi_window",      14)

    close  = df["close"]
    high   = df["high"]
    low    = df["low"]
    volume = df["volume"]

    # ----- RSI -----
    df["rsi"] = ta.momentum.RSIIndicator(
        close, window=rsi_window
    ).rsi()

    # ----- MACD -----
    macd_ind = ta.trend.MACD(
        close,
        window_slow=macd_slow,
        window_fast=macd_fast,
        window_sign=macd_signal,
    )
    df["macd"]        = macd_ind.macd()
    df["macd_signal"] = macd_ind.macd_signal()
    df["macd_hist"]   = macd_ind.macd_diff()

    # ----- Momentum -----
    df["momentum"] = close - close.shift(mom_window)

    # ----- Stochastic -----
    lowest_low   = low.rolling(window=stoch_window).min()
    highest_high = high.rolling(window=stoch_window).max()
    stoch_k = 100 * (close - lowest_low) / (highest_high - lowest_low)
    df["stoch"] = stoch_k.rolling(window=stoch_smooth).mean()

    # ----- VW-MACD (VWMA fast - VWMA slow) -----
    vwma_fast = (close * volume).rolling(vwma_fast_win).sum() / volume.rolling(vwma_fast_win).sum()
    vwma_slow = (close * volume).rolling(vwma_slow_win).sum() / volume.rolling(vwma_slow_win).sum()
    df["vwmacd"] = vwma_fast - vwma_slow

    # ----- CMF -----
    df["cmf"] = ta.volume.ChaikinMoneyFlowIndicator(
        high, low, close, volume, window=cmf_window
    ).chaikin_money_flow()

    # ----- CCI -----
    df["cci"] = ta.trend.CCIIndicator(
        high, low, close,
        window=cci_window, constant=0.015, fillna=False
    ).cci()

    # ----- MFI -----
    df["mfi"] = ta.volume.MFIIndicator(
        high, low, close, volume,
        window=mfi_window, fillna=False
    ).money_flow_index()

    # ----- OBV (نفس الكود القديم) -----
    obv_values = [0.0]
    for i in range(1, len(df)):
        if close.iloc[i] > close.iloc[i - 1]:
            obv_values.append(obv_values[-1] + volume.iloc[i])
        elif close.iloc[i] < close.iloc[i - 1]:
            obv_values.append(obv_values[-1] - volume.iloc[i])
        else:
            obv_values.append(obv_values[-1])
    df["obv"] = obv_values

    # Fill NaNs
    df = df.fillna(method="bfill").fillna(method="ffill")

    return df

#-------------------------------------------
# 2) Fast performance evaluator (بدون طباعة) + drawdown per trade
#-------------------------------------------

def quick_performance(df):
    """
    - يحسب P&L لكل صفقة
    - يحسب max drawdown% لكل صفقة (أكبر نزول عن سعر الدخول)
    - يرجّع ملخّص: total_trades, win_rate, avg_pnl, total_return,
                    max_dd, avg_dd, median_dd
    """
    entries = df[df["signal"] == 1].copy()
    exits   = df[df["signal"] == -1].copy()

    if len(entries) == 0:
        return {
            "total_trades": 0,
            "win_rate": 0.0,
            "avg_pnl": 0.0,
            "total_return": 0.0,
            "max_dd": 0.0,
            "avg_dd": 0.0,
            "median_dd": 0.0,
        }

    pnl_list = []
    dd_list  = []

    for i, entry_row in entries.iterrows():
        entry_idx   = df.index.get_loc(i)
        entry_price = entry_row["entry_price"]

        # نلاقي أول exit بعد entry
        exit_idx = None
        exit_price = None
        for j, exit_row in exits.iterrows():
            candidate_idx = df.index.get_loc(j)
            if candidate_idx > entry_idx:
                exit_idx   = candidate_idx
                exit_price = exit_row["exit_price"]
                break

        # لو مفيش exit -> نخرج عند آخر شمعة
        if exit_idx is None:
            exit_idx   = len(df) - 1
            exit_price = df["close"].iloc[-1]

        # --- P&L للصفقة ---
        pnl_pct = (exit_price - entry_price) / entry_price * 100.0
        pnl_list.append(pnl_pct)

        # --- Drawdown للصفقة ---
        # ناخد كل الشموع من الدخول لحد الخروج
        trade_slice = df.iloc[entry_idx:exit_idx + 1]
        # أسوأ نزول بالنسبة لسعر الدخول بناءً على الـ low
        dd_series = (trade_slice["low"] - entry_price) / entry_price * 100.0
        worst_dd = dd_series.min()  # ده رقم سالب غالباً
        dd_mag   = abs(worst_dd)    # نخليه موجب: 7% drawdown بدل -7%
        dd_list.append(dd_mag)

    trades = np.array(pnl_list)
    dds    = np.array(dd_list)

    total_trades = len(trades)
    wins   = (trades > 0).sum()
    win_rate = (wins / total_trades * 100.0) if total_trades > 0 else 0.0
    avg_pnl = trades.mean() if total_trades > 0 else 0.0
    total_return = trades.sum()

    max_dd     = dds.max()     if len(dds) > 0 else 0.0
    avg_dd     = dds.mean()    if len(dds) > 0 else 0.0
    median_dd  = np.median(dds) if len(dds) > 0 else 0.0

    return {
        "total_trades": int(total_trades),
        "win_rate": float(win_rate),
        "avg_pnl": float(avg_pnl),
        "total_return": float(total_return),
        "max_dd": float(max_dd),
        "avg_dd": float(avg_dd),
        "median_dd": float(median_dd),
    }

#-------------------------------------------
# 3) Pipeline for one parameter set
#    (مع لعب في indicators ON/OFF + pivot period)
#-------------------------------------------

def run_pipeline_with_params(csv_filepath, params):
    """
    - يحمّل الداتا
    - يحسب المؤشرات بالـ windows الجديدة
    - يظبط Config:
        * pivot period (prd)
        * أي indicators عايز تطفيها/تشغّلها
    - يطلع pivots + divergences
    - يشغّل الاستراتيجية
    - يرجع metrics + df الناتج
    """
    # نخزن الـ state الأصلي عشان نرجّعه بعدين
    orig_state = {
        "prd":       config.prd,
        "calcmacd":  config.calcmacd,
        "calcmacda": config.calcmacda,
        "calcrsi":   config.calcrsi,
        "calcstoc":  config.calcstoc,
        "calccci":   config.calccci,
        "calcmom":   config.calcmom,
        "calcobv":   config.calcobv,
        "calcvwmacd":config.calcvwmacd,
        "calccmf":   config.calccmf,
        "calcmfi":   config.calcmfi,
    }

    try:
        # 1) نغيّر الـ pivot period لو موجود في params
        pivot_prd = params.get("pivot_prd", config.prd)
        config.prd = int(pivot_prd)

        # 2) نطبّق ON/OFF للمؤشرات لو params فيها flags
        indicator_flags = {
            "use_macd":      "calcmacd",
            "use_macd_hist": "calcmacda",
            "use_rsi":       "calcrsi",
            "use_stoch":     "calcstoc",
            "use_mom":       "calcmom",
            "use_cci":       "calccci",
            "use_obv":       "calcobv",
            "use_vwmacd":    "calcvwmacd",
            "use_cmf":       "calccmf",
            "use_mfi":       "calcmfi",
        }
        for param_key, cfg_attr in indicator_flags.items():
            if param_key in params:
                setattr(config, cfg_attr, bool(params[param_key]))

        # 3) Pipeline عادي
        df = load_data_from_csv(csv_filepath)
        df = calculate_indicators_with_params(df, params)
        df = find_pivots(df, config.prd)
        df = scan_all_divergences(df)
        df = run_strategy(df)
        metrics = quick_performance(df)
    finally:
        # نرجّع Config زي ما كانت عشان التجربة اللي بعدها
        config.prd       = orig_state["prd"]
        config.calcmacd  = orig_state["calcmacd"]
        config.calcmacda = orig_state["calcmacda"]
        config.calcrsi   = orig_state["calcrsi"]
        config.calcstoc  = orig_state["calcstoc"]
        config.calccci   = orig_state["calccci"]
        config.calcmom   = orig_state["calcmom"]
        config.calcobv   = orig_state["calcobv"]
        config.calcvwmacd= orig_state["calcvwmacd"]
        config.calccmf   = orig_state["calccmf"]
        config.calcmfi   = orig_state["calcmfi"]

    return metrics, df

#-------------------------------------------
# 4) Grid search على windows + indicators + pivot period
#-------------------------------------------

# تقدر توسّع الـ grid براحتك لحد ~4000 combination زي ما انت قولت
param_grid = {
    # ---- indicator windows ----
    # كان [7, 14] بس → خلّيه [7, 10, 14, 21]  (4 قيم بدل 2)
    "rsi_window"   : [7, 10, 14, 21],

    # كان [9] بس → خلّيه [9, 14]  (2 قيم بدل 1)
    "stoch_window" : [9, 14],

    "stoch_smooth" : [3],
    "mom_window"   : [5, 10],
    "cci_window"   : [10],
    "mfi_window"   : [10, 20],
    "vwma_fast"    : [8, 12],
    "vwma_slow"    : [26],
    "cmf_window"   : [14, 21],

    # ---- pivot period ----
    "pivot_prd"    : [3, 5],

    # ---- indicator ON/OFF ----
    "use_vwmacd"   : [True, False],
    "use_cmf"      : [True, False],
    "use_mfi"      : [True, False],
}

grid = list(ParameterGrid(param_grid))
print(f"🔍 Number of combinations to test: {len(grid)}")

results = []

for idx, params in enumerate(grid, start=1):
    metrics, _ = run_pipeline_with_params(csv_filename, params)
    row = {**params, **metrics}
    results.append(row)

    # ✅ طباعة كاملة لكل combination
    print(f"\n[{idx}/{len(grid)}]")
    print(f"  params : {params}")
    print(
        "  metrics: "
        f"trades={metrics['total_trades']}, "
        f"win_rate={metrics['win_rate']:.2f}%, "
        f"avg_pnl={metrics['avg_pnl']:.2f}%, "
        f"total_return={metrics['total_return']:.2f}%, "
        f"max_dd={metrics['max_dd']:.2f}%, "
        f"avg_dd={metrics['avg_dd']:.2f}%, "
        f"median_dd={metrics['median_dd']:.2f}%"
    )

    # 💾 checkpoint كل 100 combo عشان الأمان
    if idx % 100 == 0 or idx == len(grid):
        tmp_df = pd.DataFrame(results)
        tmp_df.to_csv("divergence_param_grid_results_partial.csv", index=False)
        print("  💾 checkpoint saved: divergence_param_grid_results_partial.csv")


results_df = pd.DataFrame(results)

# نرتب حسب:
#   1) أعلى Total Return
#   2) أعلى Win Rate
#   3) أقل Max Drawdown
results_df = results_df.sort_values(
    by=["total_return", "win_rate", "max_dd"],
    ascending=[False, False, True]
).reset_index(drop=True)

print("\n===============================================")
print("🏆 TOP 10 PARAMETER SETS (by total_return, win_rate, low max_dd)")
print("===============================================")
print(results_df.head(10))
#-------------------------------------------
# 5) Export full grid-search results to CSV
#-------------------------------------------

csv_output_path = "divergence_param_grid_results.csv"
results_df.to_csv(csv_output_path, index=False)
print(f"\n📁 Full grid-search results saved to: {csv_output_path}")

best_params = results_df.iloc[0][list(param_grid.keys())].to_dict()
print("\n🔥 BEST PARAMS FOUND:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# شغّل الاستراتيجية تاني على أحسن params وشوف الشارت:
best_metrics, best_df = run_pipeline_with_params(csv_filename, best_params)
print("\nBest metrics on full sample:")
print(best_metrics)

# ترسم الشارت باستخدام best_df:
# plot_candlestick_with_signals(best_df)


🔍 Number of combinations to test: 2048
Loading data from PLD.csv...
Loaded 20657 bars from 2025-01-01 18:15:00-05:00 to 2025-11-14 16:45:00-05:00
Finding pivot points with period 3...
Found 2127 pivot highs and 2142 pivot lows
Scanning for divergences...
Found 2361 bars with divergences
Running strategy...

[1/2048]
  params : {'cci_window': 10, 'cmf_window': 14, 'mfi_window': 10, 'mom_window': 5, 'pivot_prd': 3, 'rsi_window': 7, 'stoch_smooth': 3, 'stoch_window': 9, 'use_cmf': True, 'use_mfi': True, 'use_vwmacd': True, 'vwma_fast': 8, 'vwma_slow': 26}
  metrics: trades=266, win_rate=73.68%, avg_pnl=0.16%, total_return=43.67%, max_dd=7.98%, avg_dd=0.92%, median_dd=0.61%
Loading data from PLD.csv...
Loaded 20657 bars from 2025-01-01 18:15:00-05:00 to 2025-11-14 16:45:00-05:00
Finding pivot points with period 3...
Found 2127 pivot highs and 2142 pivot lows
Scanning for divergences...
Found 2410 bars with divergences
Running strategy...

[2/2048]
  params : {'cci_window': 10, 'cmf_window'

KeyboardInterrupt: 